In [13]:
import easyocr
import numpy as np
import cv2
import pandas as pd
import string
import re
from collections import Counter
from nltk.corpus import stopwords
import pandas as pd
import cv2
#import pytesseract
from pytesseract import Output
import re
import matplotlib.pyplot as plt
import pickle

from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from transformers import BertTokenizer, BertModel
import torch
from sklearn.cluster import KMeans 
import base64
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

[nltk_data] Downloading package stopwords to /Users/alex/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


- Menu 1: Asian menu, "menu_2p2"
- Menu 2: Olive Garden, "olive-garden-menu-6"
- Menu 3: Red Lobster, "Red-Lobster-menu-1"
- Menu 4: Red Lobster, "Red-Lobster-menu-2"
- Menu 5: Red Lobster, "Red-Lobster-menu-5"
- Menu 6: Mexican menu,'mexican-menu-1'
- Menu 7: Mexican menu, 'mexican-menu-2'
- Menu 8: Mexican menu,'mexican-menu-3',
- Menu 9: Mexican menu,'mexican-menu-4',
- Menu 10: American menu,'american-menu-1',
- Menu 11: American menu,'american-menu-2'
- Menu 12: Random menu,'ivy-menu-1'
- Menu 13: Random menu,'ivy-menu-2'

Make another feature indicating the type of cuisine? This might be something the user would input to help the model make more accurate classification. 

In [3]:
reader = easyocr.Reader(['en'])

Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.


In [4]:
reader

In [3]:
df_bert = pd.read_csv('Combined_menu_23_items_bert.csv')
#df = pd.read_csv('Combined_menu_23_items.csv')
#df.drop('Unnamed: 0', axis = 1, inplace = True)
df_bert.drop('Menu',axis = 1,inplace= True)
df_bert.drop('area',axis = 1,inplace= True)
df_bert.drop('center_x',axis = 1,inplace= True)
df_bert.drop('center_y',axis = 1,inplace= True)
df_bert.drop('common_words',axis = 1,inplace= True)
df_bert.drop('contains_dish',axis = 1,inplace= True)
df_bert.drop('non_words',axis = 1,inplace= True)
df_bert.drop('stop_words',axis = 1,inplace= True)
df_bert.drop('prepositions',axis = 1,inplace= True)
#############
#df_bert.drop('numbers',axis = 1,inplace= True)
#df_bert.drop('commas',axis = 1,inplace= True)
# df_bert.drop('exclamation',axis = 1,inplace= True)
# df_bert.drop('question',axis = 1,inplace= True)
# df_bert.drop('colons',axis = 1,inplace= True)
# df_bert.drop('underscores',axis = 1,inplace= True)
# df_bert.drop('dollar',axis = 1,inplace= True)
#df_bert.drop('punctuation',axis = 1,inplace= True)

#df.drop('width',axis = 1,inplace= True)
#df.drop('height',axis = 1,inplace= True)

In [174]:
df = pd.read_csv('Combined_menu_23_items_bert.csv')

In [14]:
file = 'test_menu.jpeg'

image = cv2.imread(file)

# Convert the image to RGB (OpenCV loads images in BGR format)
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

# Convert the image array to a byte stream
_, buffer = cv2.imencode('.png', image)

# Convert the byte stream to a base64 string
image_base64 = base64.b64encode(buffer).decode('utf-8')

In [195]:
list_of_items = df[df.Item == 1]['Text'].str.replace('[^a-zA-Z ]', '', regex=True).str.strip().to_frame()
list_of_items['Type'] = 5*np.ones(len(list_of_items)).astype('int32')
list_of_items.rename(columns={"Text": "Food"})
list_of_items.reset_index(drop = True, inplace = True)
list_of_items.to_csv('food_list.csv', index = False)
#Drinks, Appetizers, Main, Salads, Soups, Desserts

### Decision Tree

In [325]:
regression_tree = DecisionTreeClassifier(criterion = "gini")
regression_tree.fit(X_train, y_train)
score = regression_tree.score(X_test, y_test)
y_hat_tree = regression_tree.predict(X_test)

print('Score: ', score)

Score:  0.8576972833117723


### Random Forest

In [326]:
rfc = RandomForestClassifier(bootstrap=True, n_estimators=200,random_state=0)
rfc.fit(X_train, y_train)
score = rfc.score(X_test, y_test)
y_hat_tree = rfc.predict(X_test)

print('Score: ', score)

/var/folders/mq/z3r_z9nd59g6z5rfmlxsltg40000gn/T/ipykernel_61498/467280947.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rfc.fit(X_train, y_train)


Score:  0.8602846054333765


## Train Test Split

In [15]:
df_bert_test.head()

,Text,width,height,uppercase,chars,words,periods,period_btw_numbers,number_end,numbers,commas,exclamation,question,colons,underscores,dollar,punctuation,2_periods_cnt,Item,vector_0,vector_1,vector_2,vector_3,vector_4,vector_5,vector_6,vector_7,vector_8,vector_9,vector_10,vector_11,vector_12,vector_13,vector_14,vector_15,vector_16,vector_17,vector_18,vector_19,vector_20,vector_21,vector_22,vector_23,vector_24,vector_25,vector_26,vector_27,vector_28,vector_29,vector_30,vector_31,vector_32,vector_33,vector_34,vector_35,vector_36,vector_37,vector_38,vector_39,vector_40,vector_41,vector_42,vector_43,vector_44,vector_45,vector_46,vector_47,vector_48,vector_49,vector_50,vector_51,vector_52,vector_53,vector_54,vector_55,vector_56,vector_57,vector_58,vector_59,vector_60,vector_61,vector_62,vector_63,vector_64,vector_65,vector_66,vector_67,vector_68,vector_69,vector_70,vector_71,vector_72,vector_73,vector_74,vector_75,vector_76,vector_77,vector_78,vector_79,vector_80,vector_81,vector_82,vector_83,vector_84,vector_85,vector_86,vector_87,vector_88,vector_89,vector_90,vector_91,vector_92,vector_93,vector_94,vector_95,vector_96,vector_97,vector_98,vector_99,vector_100,vector_101,vector_102,vector_103,vector_104,vector_105,vector_106,vector_107,vector_108,vector_109,vector_110,vector_111,vector_112,vector_113,vector_114,vector_115,vector_116,vector_117,vector_118,vector_119,vector_120,vector_121,vector_122,vector_123,vector_124,vector_125,vector_126,vector_127,vector_128,vector_129,vector_130,vector_131,vector_132,vector_133,vector_134,vector_135,vector_136,vector_137,vector_138,vector_139,vector_140,vector_141,vector_142,vector_143,vector_144,vector_145,vector_146,vector_147,vector_148,vector_149,vector_150,vector_151,vector_152,vector_153,vector_154,vector_155,vector_156,vector_157,vector_158,vector_159,vector_160,vector_161,vector_162,vector_163,vector_164,vector_165,vector_166,vector_167,vector_168,vector_169,vector_170,vector_171,vector_172,vector_173,vector_174,vector_175,vector_176,vector_177,vector_178,vector_179,vector_180,vector_181,vector_182,vector_183,vector_184,vector_185,vector_186,vector_187,vector_188,vector_189,vector_190,vector_191,vector_192,vector_193,vector_194,vector_195,vector_196,vector_197,vector_198,vector_199,vector_200,vector_201,vector_202,vector_203,vector_204,vector_205,vector_206,vector_207,vector_208,vector_209,vector_210,vector_211,vector_212,vector_213,vector_214,vector_215,vector_216,vector_217,vector_218,vector_219,vector_220,vector_221,vector_222,vector_223,vector_224,vector_225,vector_226,vector_227,vector_228,vector_229,vector_230,vector_231,vector_232,vector_233,vector_234,vector_235,vector_236,vector_237,vector_238,vector_239,vector_240,vector_241,vector_242,vector_243,vector_244,vector_245,vector_246,vector_247,vector_248,vector_249,vector_250,vector_251,vector_252,vector_253,vector_254,vector_255,vector_256,vector_257,vector_258,vector_259,vector_260,vector_261,vector_262,vector_263,vector_264,vector_265,vector_266,vector_267,vector_268,vector_269,vector_270,vector_271,vector_272,vector_273,vector_274,vector_275,vector_276,vector_277,vector_278,vector_279,vector_280,vector_281,vector_282,vector_283,vector_284,vector_285,vector_286,vector_287,vector_288,vector_289,vector_290,vector_291,vector_292,vector_293,vector_294,vector_295,vector_296,vector_297,vector_298,vector_299,vector_300,vector_301,vector_302,vector_303,vector_304,vector_305,vector_306,vector_307,vector_308,vector_309,vector_310,vector_311,vector_312,vector_313,vector_314,vector_315,vector_316,vector_317,vector_318,vector_319,vector_320,vector_321,vector_322,vector_323,vector_324,vector_325,vector_326,vector_327,vector_328,vector_329,vector_330,vector_331,vector_332,vector_333,vector_334,vector_335,vector_336,vector_337,vector_338,vector_339,vector_340,vector_341,vector_342,vector_343,vector_344,vector_345,vector_346,vector_347,vector_348,vector_349,vector_350,vector_351,vector_352,vector_353,vector_354,vector_355,vector_356,vector_357,

In [16]:
X = df_bert_test.drop(['Item','Text'],axis = 1)
y = df_bert_test[['Item']]

test_size = 0.3
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = test_size, random_state=0)

## XGBoost

### GridSearch

In [368]:
xgb = XGBClassifier(random_state=0)


param_grid = {
    'n_estimators': [50, 100, 200, 300],
    'max_depth': [2, 4, 6, 8],
    'learning_rate': [0.01,0.05, 0.1, 0.2]
}

grid_search = GridSearchCV(xgb, param_grid, cv=3, scoring='accuracy')

# Fit the RandomizedSearchCV
grid_search.fit(X_train, y_train)

# Get the best parameters

print("Best parameters: ", grid_search.best_params_)
print("Best score: ", grid_search.best_score_)


Best parameters:  {'learning_rate': 0.2, 'max_depth': 6, 'n_estimators': 300}
Best score:  0.9223516361619523


### Model


150, 0.3

In [35]:
xgb = XGBClassifier(n_estimators= 150,learning_rate = 0.3,random_state=0)
xgb.fit(X_train, y_train)
score = xgb.score(X_test, y_test)
y_hat_tree = xgb.predict(X_test)

print('XGBoost Score: ', score)

XGBoost Score:  0.906856403622251


In [157]:
with open('xgb.pkl', 'wb') as f:
    pickle.dump(xgb, f)

In [34]:
xgb = pickle.load(open('xgb.pkl', "rb"))

In [31]:
df_y_test = y_test
df_y_test.rename({'Item': 'Actual'}, axis=1, inplace = True)
df_y_test['Predicted'] = y_hat_tree
df_y_test.sort_index(axis = 0, inplace = True)
df_test = df[df.index.isin(df_y_test.index)].drop('Item',axis = 1)
df_test = pd.concat([df_test, df_y_test], axis=1)
df_test[['Menu','Text','height','Actual','Predicted']]

,Menu,Text,height,Actual,Predicted
1,1,16,109,0,0
4,1,spicy & sour w/ mushrooms,120,0,0
6,1,in lemon grass broth,128,0,0
9,1,L6.95,99,0,0
10,1,18,110,0,0
11,1,Tom Kha Kai (chicken),134,1,1
14,1,L5.95,110,0,0
17,1,19,110,0,0
18,1,Soft Tofu Soup,149,1,1
22,1,thread noodles w/ vegetables,138,0,0


## Test model with a new menu

In [6]:
# Load the list from the file
with open('dish_items.pkl', 'rb') as f:
    dish_items = pickle.load(f)
#print(dish_items)

In [7]:
# Load the list from the file
with open('common_menu_words.pkl', 'rb') as f:
    common_menu_words = pickle.load(f)
#print(common_menu_words)

In [2]:
start_menu = 1

# stop_words = set(stopwords.words('english'))

# prepositions = ['aboard', 'across', 'along', 'amid', 'among', 'around', 'behind', 'beneath', 'beside', 'beyond', \
#                 'concerning', 'considering', 'despite', 'except', 'inside', 'like', 'near', 'onto', 'outside', 'past',\
#                 'regarding', 'round', 'since', 'throughout', 'toward', 'underneath', 'unto', 'upon', 'within', 'without']

# food_categories = ['gyros','sandwiches','toppings','sauces','orders','shareables','drinks', 'beverages', 'appetizers',\
#                    'starters', 'salads', 'soups', 'entrees','entrées', 'main courses', 'desserts', 'sweets', 'sandwiches',\
#                    'pizzas', 'burgers', 'pastas', 'seafood', 'steaks', 'vegetarian/vegan', 'sides', 'side dishes',\
#                    'breakfast/brunch', "kids' menu", 'combos', 'platters', 'specials of the day', 'snacks', 'bar snacks',\
#                    'sushi', 'japanese cuisine', 'dim sum', 'chinese cuisine', 'tacos', 'mexican cuisine', 'tapas',\
#                    'spanish cuisine', 'mezze', 'mediterranean cuisine', 'bento', 'japanese lunch boxes', 'cuisines',\
#                    'wraps/rolls', 'all you can eat', 'family style', 'shared dishes','sauce']

# non_words = ['available',"choice","choices","add","oz","available","w",'served','side','additional','contains','allergy','.com',\
#             'pizzeria','salads','soups','side items','appetizers','create','classics','your own','plates','extra']

# non_words += food_categories

def count_uppercase_letters(text):
    return sum(1 for i in text if i.isupper())
def count_numerical_chars(text):
    return sum(1 for i in text if i.isdigit())
def count_punctuation(text):
    return sum(1 for i in text if i in string.punctuation and i not in [',', '.', '$',':'])
def count_common_menu_words(text):
    words = re.findall(r'\b[a-zA-Z]+\b', text)  # Extract words containing only alphabetical characters
    lowercase_words = [word.lower() for word in words]  # Convert all words to lowercase
    return sum(word in common_menu_words for word in lowercase_words)
def count_consecutive_periods(text):
    return len(re.findall(r'\.{2,}', text))
# def non_words_count(text):
#     words = re.findall(r'\b[a-zA-Z]+\b', text)  # Extract words containing only alphabetical characters
#     lowercase_words = [word.lower() for word in words]  # Convert all words to lowercase
#     return sum(lowercase_words.count(non_word) for non_word in non_words)
# def stop_words_count(text):
#     words = re.findall(r'\b[a-zA-Z]+\b', text)  # Extract words containing only alphabetical characters
#     lowercase_words = [word.lower() for word in words]  # Convert all words to lowercase
#     return sum(lowercase_words.count(stop_word) for stop_word in stop_words)
# def prepositions_count(text):
#     words = re.findall(r'\b[a-zA-Z]+\b', text)  # Extract words containing only alphabetical characters
#     lowercase_words = [word.lower() for word in words]  # Convert all words to lowercase
#     return sum(lowercase_words.count(preposition) for preposition in prepositions)

# Create a reader for English language
reader = easyocr.Reader(['en'])

files = ['test_menu.jpeg']

for c,file in enumerate(files):    
    # Read the images using OpenCV
    image = cv2.imread(file)
    
    # Convert the image to RGB (OpenCV loads images in BGR format)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    # Get the dimensions of the image
    height_img, width_img, _ = image.shape
    area_img = width_img*height_img
    
    # Perform OCR on the image
    result = reader.readtext(image)

    # Prepare a list to store the data
    data = []

    # Loop through the results
    for (bbox, text, prob) in result:
        # unpack the bounding box
        (tl, tr, br, bl) = bbox
        tl = (int(tl[0]), int(tl[1]))
        tr = (int(tr[0]), int(tr[1]))
        br = (int(br[0]), int(br[1]))
        bl = (int(bl[0]), int(bl[1]))        
        
        # calculate the center coordinates
        center_x = int((tl[0] + tr[0] + br[0] + bl[0]) / 4)/(width_img)
        center_y = int((tl[1] + tr[1] + br[1] + bl[1]) / 4)/(height_img)

        # Compute the width and height of the bounding box
        width = np.sqrt((br[0]-bl[0])**2 + (br[1]-bl[1])**2)/(width_img)
        height = np.sqrt((tr[0]-br[0])**2 + (tr[1]-br[1])**2)/(height_img)

        # Compute the area of the bounding box
        area = (width * height)

        # Append the text, probability, bbox coordinates, width, height, and area to the data list
        data.append([text, width, height])

    # data_2 = [(entry[0], entry[6], entry[7], entry[8]) for entry in data]
    
    # Create a pandas DataFrame from the data
    df = pd.DataFrame(data, columns=["Text" , "width", "height"])

    # Select only 'Text' and 'Height' columns
#    df.insert(0, 'Menu', c + start_menu)
    df = df[['Text', "width", 'height']]
#     df = df.astype({'width':'int'})
#     df = df.astype({'height':'int'})
#     df = df.astype({'area':'int'})
    df['uppercase'] = df['Text'].apply(count_uppercase_letters)

    df['Text'] = df['Text'].str.strip()
    # Add character count
    df['chars'] = df['Text'].apply(len)

    # Add word count
    df['words'] = df['Text'].apply(lambda x: len(x.split()))
    
#     df['contains_dish'] = df['Text'].apply(lambda x: 1 if any(item in x for item in dish_items) else 0)

#     df['common_words'] = df['Text'].apply(count_common_menu_words)    
#     df['non_words'] = df['Text'].apply(lambda x: non_words_count(x))

    df['periods'] = df['Text'].apply(lambda x: x.count('.'))
    df['period_btw_numbers'] = df['Text'].apply(lambda x: bool(re.search(r'\d\.\d', x))).astype(int)
    df['number_end'] = df['Text'].apply(lambda x: bool(re.search(r'\d$', x))).astype(int)
    df['numbers'] = df['Text'].apply(count_numerical_chars)
    df['commas'] = df['Text'].apply(lambda x: x.count(','))
    df['exclamation'] = df['Text'].apply(lambda x: x.count('!'))
    df['question'] = df['Text'].apply(lambda x: x.count('?'))
    df['colons'] = df['Text'].apply(lambda x: x.count(':'))
    df['underscores'] = df['Text'].apply(lambda x: x.count('_'))
    df['dollar'] = df['Text'].apply(lambda x: x.count('$'))
    df['punctuation'] = df['Text'].apply(count_punctuation)
    df['2_periods_cnt'] = df['Text'].apply(count_consecutive_periods)
#     df['stop_words'] = df['Text'].apply(stop_words_count)
#     df['prepositions'] = df['Text'].apply(prepositions_count)

    df['Item'] = np.zeros(len(df))
    df['Item'] = df['Item'].astype('int')
    if c > 0:
        dfs = pd.concat([dfs,df], ignore_index=True)
    else:
        dfs = df.copy()
dfs

Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.


,Text,width,height,uppercase,chars,words,periods,period_btw_numbers,number_end,numbers,commas,exclamation,question,colons,underscores,dollar,punctuation,2_periods_cnt,Item
0,roanoke,0.377336,0.059701,0,7,1,0,0,0,0,0,0,0,0,0,0,0,0,0
1,BEVERAGES,0.205607,0.024876,9,9,1,0,0,0,0,0,0,0,0,0,0,0,0,0
2,COFFEE,0.112150,0.021559,6,6,1,0,0,0,0,0,0,0,0,0,0,0,0,0
3,CAFE LATTE,0.175234,0.023217,9,10,2,0,0,0,0,0,0,0,0,0,0,0,0,0
4,BLOOD ORANGE,0.228972,0.021559,11,12,2,0,0,0,0,0,0,0,0,0,0,0,0,0
5,intelligentsia,0.149533,0.023217,0,14,1,0,0,0,0,0,0,0,0,0,0,0,0,0
6,"flavored vanilla, hazelnut",0.282710,0.023217,0,26,3,0,0,0,0,1,0,0,0,0,0,0,0,0
7,MIMOSA,0.116822,0.021559,6,6,1,0,0,0,0,0,0,0,0,0,0,0,0,0
8,11,0.035047,0.019900,0,2,1,0,0,1,2,0,0,0,0,0,0,0,0,0
9,ESPRESSO,0.151869,0.019900,8,8,1,0,0,0,0,0,0,0,0,0,0,0,0,0


In [4]:
with open('data.pkl', 'wb') as f:
    pickle.dump(data, f)

## BERT Conversion

In [5]:
model = BertModel.from_pretrained('bert-base-uncased')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

def texts_to_vectors(texts):
    print("Starting texts_to_vectors function...")
    
    vectors = []
    for cnt, text in enumerate(texts):
        try:            
            inputs = tokenizer([text], return_tensors="pt", truncation=True, padding=True)
            with torch.no_grad():
                outputs = model(**inputs)
            embeddings = outputs.last_hidden_state
            mask = inputs.attention_mask
            masked_embeddings = embeddings * mask.unsqueeze(-1)
            summed = torch.sum(masked_embeddings, 1)
            summed_mask = torch.clamp(mask.sum(1), min=1e-9)
            mean_pooled = summed / summed_mask.unsqueeze(-1)
            vectors.append(mean_pooled[0].numpy())
        except Exception as e:
            print(f"Error encountered while processing text {cnt}: {e}")
            continue
        # Print progress update every 400 samples
        if cnt % 400 == 0:
            percentage = (cnt / len(texts)) * 100
            print(f"Processing text {cnt} of {len(texts)} ({percentage:.2f}% complete)")
    
    print("Finished converting texts to vectors.")
    return np.array(vectors)

vectors = texts_to_vectors(dfs['Text'].tolist())

df_vectors = pd.DataFrame(vectors, columns=[f'vector_{i}' for i in range(vectors.shape[1])])
df_bert_test = pd.concat([dfs, df_vectors], axis=1)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [12]:
dfs = df_bert[['Text','width','height','uppercase','chars','words','periods','period_btw_numbers','number_end','numbers',\
 'commas','exclamation','question','colons','underscores','dollar','punctuation','2_periods_cnt','Item']]#.head()

### Predict

In [32]:
df_predict = df_bert_test.drop(['Item','Text'],axis = 1)
                           #     ,'area','center_x','center_y','common_words','contains_dish',\
                          #      'non_words','stop_words','prepositions'],axis = 1)
#,'center_x','center_y','area'
item = xgb.predict(df_predict)
df_menu = df_bert_test.copy()
df_menu['Item'] = item

df_menu[['Text','Item']]

# list_of_items = df_menu[df_menu.Item == 1]['Text'].str.replace('[^a-zA-Z ]', '', regex=True).str.strip().tolist()

# list_of_items

,Text,Item
0,roanoke,0
1,BEVERAGES,0
2,COFFEE,1
3,CAFE LATTE,1
4,BLOOD ORANGE,1
5,intelligentsia,0
6,"flavored vanilla, hazelnut",0
7,MIMOSA,1
8,11,0
9,ESPRESSO,1


In [97]:
items_df = pd.DataFrame(list_of_items, columns = ['Item'])

In [121]:
df_menu[df_menu.Item == 1].drop(columns = ['Text'])

Menu  center_x  center_y     width    height      area  uppercase  chars  \
2      1  0.109813  0.201493  0.112150  0.021559  0.002418          6      6   
3      1  0.468458  0.200663  0.175234  0.023217  0.004068          9     10   
4      1  0.824766  0.201493  0.228972  0.021559  0.004936         11     12   
7      1  0.768692  0.223051  0.116822  0.021559  0.002519          6      6   
9      1  0.128505  0.252902  0.151869  0.019900  0.003022          8      8   
12     1  0.542056  0.254561  0.154206  0.021559  0.003324          5      9   
21     1  0.357477  0.359038  0.383178  0.021559  0.008261         17     22   
28     1  0.342290  0.485904  0.581776  0.025705  0.014954         27     35   
31     1  0.258178  0.558872  0.413551  0.023217  0.009602         21     23   
34     1  0.217290  0.610282  0.331776  0.022388  0.007428         17     18   
38     1  0.211449  0.684080  0.321262  0.021559  0.006926         16     18   
41     1  0.103972  0.736318  0.107477  0.019900  0.002139          6      6   
42     1  0.268692  0.737148  0.205607  0.021559  0.004433         11     12   
45     1  0.278037  0.788557  0.453271  0.021559  0.009772         22     25   
50     1  0.114486  0.899668  0.126168  0.021559  0.002720          5      7   
52     1  0.704439  0.898839  0.324766  0.023217  0.007540         17     18   
53     1  0.261682  0.930348  0.420561  0.023217  0.009764         20     22   
55     1  0.700935  0.930348  0.116822  0.019900  0.002325          5      7   
56     1  0.198598  0.961028  0.289720  0.021559  0.006246         13     16   
57     1  0.664720  0.961028  0.250000  0.021559  0.005390         13     14   
60     1  0.318925  0.982587  0.184579  0.021559  0.003979          7     11   

    words  contains_dish  common_words  non_words  periods  \
2       1              0             0          0        0   
3       2              0             0          0        0   
4       2              0             1          0        0   
7       1              0             0          0        0   
9       1              0             1          0        0   
12      2              0             0          0        1   
21      4              0             2          0        0   
28      5              0             1          0        1   
31      3              0             1          0        0   
34      2              0             2          0        0   
38      3              0             1          0        0   
41      1              0             0          0        0   
42      2              0             0          0        0   
45      4              0             0          0        0   
50      2              0             1          0        0   
52      2              0             2          0        0   
53      3              0             0          0        0   
55      2              0             0          0        0   
56      3              0             0          0        0   
57      2              0             0          0        0   
60      2              0             1          0        1   

    period_btw_numbers  number_end  numbers  commas  exclamation  question  \
2                    0           0        0       0            0         0   
3                    0           0        0       0            0         0   
4                    0           0        0       0            0         0   
7                    0           0        0       0            0         0   
9                    0           0        0       0            0         0   
12                   1           1        2       0            0         0   
21                   0           1        2       0            0         0   
28                   1           1        3       0            0         0   
31                   0           0        0       0            0         0   
34                   0           0        0       0            0         0   
38                  

### Clustering... nvm

In [126]:
n_clusters = 4

#items_df_clusters = pd.concat([items_df, df_vectors_items], axis=1)

k_means = KMeans(init = "k-means++", n_clusters = n_clusters, n_init = 30 ,random_state = 0)
k_means.fit(df_menu[df_menu.Item == 1].drop(columns = ['Text']))

df_KNN = df_menu[df_menu.Item == 1][['Text']].copy()
df_KNN['Cluster'] = k_means.labels_
df_KNN

,Text,Cluster
2,COFFEE,1
3,CAFE LATTE,3
4,BLOOD ORANGE,3
7,MIMOSA,1
9,ESPRESSO,1
12,MOCHA 4.5,1
21,BUTTER FRENCH TOAST 11,0
28,BRAISED OXTAIL TAQUITOS HUEVOS 12.5,2
31,PORCHETTA EGGS BENEDICT,0
34,BREAKFAST SANDWICH,0
